# Part 1: Solution of Non-Linear Systems

Made by:
- Haiber Alfredo Alpala
- Juan Pablo Gomez Gomez
- Juan Diego Ospina Ocampo
- Jose Fernando Portilla Roser
- David Delgado Ortiz
- Jacobo Zapata Rojas
- Maria Paula Ardila Otero
- Mariana Valencia 
- Samuel Mira Álvarez

## Inverse Kinematics of a 2-Link Planar Robot Arm

Modern robotics requires autonomous systems to perform tasks with precision. One of the fundamental problems is inverse kinematics, that is, determining the angles that the joints of a robotic arm must adopt to reach a desired position in the plane (or space). This problem gives rise to systems of nonlinear equations, whose solution requires appropriate numerical methods. 

### **Description of the problem**

Consider a planar robotic arm composed of two links of lengths $L_{1}$ and $L_{2}$, articulated by angles $\theta_{1}$ (formed by the first link with the support base) and $\theta_{2}$ (formed by the second link with respect to the first).

The position of the end of the arm in the plane (in Cartesian coordinates) is given by:
$$
x(\theta_{1}, \theta_{2}) = L_{1} \cos (\theta_{1}) + L_{2} \cos (\theta_{1} + \theta_{2}) \\
y(\theta_{1}, \theta_{2}) = L_{1} \sin (\theta_{1}) + L_{2} \sin (\theta_{1} + \theta_{2})
$$

To reach a desired point $(x_{d}, y_{d})$, the nonlinear system is considered:
$$
F_{1}(\theta_{1}, \theta_{2}) = x(\theta_{1}, \theta_{2}) - x_{d} = 0 \\
F_{2}(\theta_{1}, \theta_{2}) = y(\theta_{1}, \theta_{2}) - y_{d} = 0
$$



In [ ]:
# Import dependencies here
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

### Part 1
Implement Newton's Method to solve the resulting non-linear system. 

Here goes activity 1. 

### Part 2
Choose realistic values for $L_{1}$, $L_{2}$ and at least three final positions $(x_{d}, y_{d})$ and evaluate the results of the method.

Here goes activity 2.

### Part 3
Analyze the sensibility of the initial conditions and discuss the cases in which the method converges and difficulties in implementation.  

Here goes activity 3.

### Part 4
Visualize the final end positions and the intermediate trajectories of the arm. 

Here goes activity 4. 

### Part 5
Generalize the model to an arm with $n \geq 3$ links. In this case, the system is composed of $n$ non-linear equations and the number of solutions can increase substantially. 

Here goes activity 5. 
